# Interactive parallel computing with ipyparallel

ipyparallel is a Python package and collection of scripts for controlling clusters for Jupyter.

ipyparallel contains the following scripts:

- ipcluster - start/stop a cluster
- ipcontroller - start a scheduler
- ipengine - start an engine


## Installing/activating ipyparallel

- first install the package: `conda install ipyparallel`
- to be able to launch parallel engines from the dashboard add `c.NotebookApp.server_extensions.append('ipyparallel.nbextension')` to the file `~/.jupyter/jupyter_notebook_config.py` (may need to create it)
- then enable the IPython Clusters tab in Jupyter by typing `ipcluster nbextension enable --user` in a terminal


### <font color="red"> *Exercise*: </font>

- Install and activate ipyparallel 
- (for Windows users: alternative location for config files is `%PROGRAMDATA%\jupyter\`)
- Start as many engines as you have cores on your laptop

## How does it work?

- Launch IPython **engines** (Independent Python processes. Typically one per core)
- Create a `Client` that acts as a proxy to the engines
- Use client to launch tasks on the engines

### There are several ways to run parallel code

- **Direct interface** - access engines directly, explicitly through their identifiers
- **Load-balanced interface** - submit job to scheduler which distributes to engines depending on load

### <font color="blue"> Demo, direct interface </blue>

In [ ]:
import ipyparallel

In [ ]:
rc = ipyparallel.Client()

The `ids` attribute of Client instance shows identifiers of engines that IPython detected

In [ ]:
rc.ids

In [ ]:
dview = rc[:]
dview

In [ ]:
dview2 = rc.direct_view()
dview2

In [ ]:
rc.direct_view?

### <font color="blue"> Demo, %px magic </blue>

Use the %px (or %%px) magic to execute given python code in parallel

In [ ]:
%px?

In [ ]:
%px import os, time

In [ ]:
%px print(time.time())

In [ ]:
%px a = os.getpid()

In [ ]:
%px print(a)

%pxresult show the result of the last %px command

In [ ]:
%pxresult?

Specify list of engines to run code on using `--targets`. Supports Python slicing

In [ ]:
%%px --targets :-1
    print(os.getpid())

### <font color="red"> *Exercise: Estimating $\pi$ with parallel Monte Carlo* </font>

Let us first make a plot to illustrate what we want to do

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pylab as plt
import numpy as np

n = 5000
p = np.random.rand(n,2)
idx = np.sqrt(p[:,0]**2+p[:,1]**2) < 1
plt.plot(p[idx,0],p[idx,1],'b.') # point inside
plt.plot(p[idx==False,0],p[idx==False,1],'r.') # point outside
plt.axis([-0.1,1.1,-0.1,1.1]) 
plt.show()

### Task 1
 1. import ipyparallel
 2. create a Client object
 3. list the identifiers of the engines
 4. create a direct interface to the engines

### Task 2
 1. take the following function mc_pi
 2. run it in serial using 10$^6$ random points, and time it
 3. run it in parallel using the map_sync method of clients
 4. time it, and compare with serial execution

In [ ]:
import numpy as np
def mc_pi(n):
    count = 0
    p = np.random.rand(n,2)
    for i in range(n):
        x = p[i,0]
        y = p[i,1]
        if (x**2 + y**2) <= 1.0:
            count += 1
    return float(count)/float(n)

### <font color="green"> *Solution: Task 1* </font>

In [ ]:
import ipyparallel

In [ ]:
clients = ipyparallel.Client()
clients.ids

In [ ]:
dview = clients[:]
dview

### <font color="green"> *Solution: Task 2* </font>

In [ ]:
nrandpts = 10**6 

In [ ]:
%%timeit
est_pi = mc_pi(nrandpts)
print(est_pi*4) 

May need to import modules on the engines  
(another approach would be to put `import numpy as np` inside `mc_pi` function)

In [ ]:
dview.execute("import numpy as np")

In [ ]:
dview.map_sync?

In [ ]:
dview.map?

We need to pass a list as second argument to `map_sync`, containing number of random points to be assigned to each process

In [ ]:
%%timeit
results = dview.map_sync(mc_pi, [nrandpts/2]*2)
print(np.sum(results))

Speed up of factor 2 on my macbook (regardless of number of started engines), since two physical cores 